In [1]:
import os

In [2]:
# Store your API keys in a common file and reference it here
from apikeys import OPEN_AI_KEY, DEEP_LAKE_KEY, GOOGLE_API_KEY, GOOGLE_CSE_ID

In [3]:
os.environ["OPENAI_API_KEY"] = OPEN_AI_KEY

## The LLMs

With LangChain, we can call a LLM on our prompt, we are going to do that right now. I am going to ask the LLM to give me a top 10 tips for efficiently managing time. 

In [4]:
from langchain.llms import OpenAI

The `temperature` parameter determines the randomness, `0` is stable and `1` is random. Creative tasks should be in the range of `0.7` and `0.9`. 

In [6]:
llm_davinci = OpenAI(model_name = "text-davinci-003", temperature= 0.9)

In [7]:
prompt = "Give me the top 10 tips for efficiently managing time"

In [9]:
tips = llm_davinci(prompt)

In [10]:
print(tips)



1. Prioritize your tasks – understand what is most important to get done and focus on completing those things first.

2. Create a plan – map out each day, week, and month so you know what needs to be accomplished and when.

3. Eliminate distractions – turn off notifications and focus on one task at a time to stay on track.

4. Tackle the difficult tasks first – doing the hard tasks when you have the most energy will help you stay motivated throughout the day.

5. Break down large projects – breaking down larger projects into smaller chunks makes them more manageable and allows for better progress tracking.

6. Utilize task management tools – there are many tools available to help manage tasks and keep track of progress.

7. Take frequent breaks – working non-stop is not effective – taking short breaks throughout the day can help you stay focused and productive.

8. Set realistic goals – setting achievable goals helps you stay motivated and focused on the tasks at hand.

9. Outsource 

## The Chains
Chains are end-to-end wrappers around components, used to create sequences. The most commonly used chain is `LLMChain`. 

`PromptTemplate` removes the overhead of repeating sentences. In any use case, we can abstract the idea into modules, take the input and provide the output. 

In [5]:
# Imports 
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [6]:
model = OpenAI(model_name = "text-davinci-003", temperature = 0.9)
prompt = PromptTemplate(
    input_variables= ["number", "product"],
    template = "Give me the top {number} places to buy {product}"
)

In [7]:
chain = LLMChain(llm = model, prompt= prompt)

In [9]:
print(prompt.format_prompt(number = "5", product = "chocolate"))

text='Give me the top 5 places to buy chocolate'


In [10]:
print(chain.run(number = "5", product = "chocolate"))



1. Lindt Chocolates
2. Godiva Chocolates
3. Ghirardelli Chocolates
4. Hershey's Chocolate World
5. Russell Stover Chocolates


## The Memory
Memory as the name suggest is the mechanism for storing and managing conversation history, to enable the AI to generate more relevant responses. 

One such method is `ConversationalBufferMemory`, it wraps around `ChatMessageHistory`, extracts messages and gives them to chain. It helps in better context-aware generation. 

In [11]:
# Imports
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [12]:
model = OpenAI(model_name = "text-davinci-003", temperature = 0.9)

In [13]:
conversation = ConversationChain(
    llm = model,
    verbose = True,
    memory = ConversationBufferMemory()
)

In [14]:
# Starting the conversation
conversation.predict(input = "Hey, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hey, how are you?
AI:


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



> Finished chain.


" Hi there! I'm doing great! It has been a productive day with lots of exciting new experiences. How about you?"

In [15]:
# Continuing the conversation 
conversation.predict(input = "What can you help me with?")
conversation.predict(input = "Appreciate it")

print(conversation)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hey, how are you?
AI:  Hi there! I'm doing great! It has been a productive day with lots of exciting new experiences. How about you?
Human: What can you help me with?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hey, how are you?
AI:  Hi there! I'm doing great! It has been a productive day with lots of exciting new experiences. How about you?
Human: What can y

Memory is basically log here. What about Context Window? When does the model forget? Questions we will be exploring. 

## Deep Lake VectorStore
As the name suggests, it is a vector store. It stores vector embeddings. This can be conversation, information or knowledge graph.

In [16]:
import os
os.environ["ACTIVELOOP_TOKEN"] = DEEP_LAKE_KEY

Install the deeplake library - `pip install deeplake`. I have run this command but the output is too long, so I am gonna cut that cell out. 

> 🧠 Do remember to `!pip install deeplake`

In [25]:
# imports
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DeepLake

In [20]:
# Model which is not at all random, very reproducable
model = OpenAI(model_name = "text-davinci-003", temperature = 0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [21]:
# Creating our documents - databases 
texts = [
    "Napoleon was defated in the Battle of Waterloo",
    "Battle of Austerlitz is considered the greatest victory of Napoleon Bonaparte",
]

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.create_documents(texts)

In [23]:
print(docs)

[Document(page_content='Napoleon was defated in the Battle of Waterloo', metadata={}), Document(page_content='Battle of Austerlitz is considered the greatest victory of Napoleon Bonaparte', metadata={})]


Recursive Character Text Splitter - Recursively Splits texts (First paragraph, then sentences, then words), the idea is to maintain the connectedness as long as possible introducing semantic meaning. 

We can now create a DeepLake dataset and upload the docs.

In [24]:
my_activeloop_org_id = "mukilan"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [26]:
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/mukilan/langchain_course_from_zero_to_hero


hub://mukilan/langchain_course_from_zero_to_hero loaded successfully.


In [27]:
# We can finally add documents to the DeepLake
db.add_documents(docs)

Evaluating ingest: 0%|                                            | 0/1 [00:00<?Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).
Evaluating ingest: 100%|██████████████████████████████████████| 1/1 [01:14<00:00
-

Dataset(path='hub://mukilan/langchain_course_from_zero_to_hero', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape     dtype  compression
  -------   -------   -------   -------  ------- 
 embedding  generic  (2, 1536)  float32   None   
    ids      text     (2, 1)      str     None   
 metadata    json     (2, 1)      str     None   
   text      text     (2, 1)      str     None   


['91a8e1c0-2b44-11ee-8634-5e6895b2d216',
 '91a8e314-2b44-11ee-8634-5e6895b2d216']

Moving on to creating a RetrievalQA Chain which will be used as tool for the upcoming agent.

In [28]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm = model,
    chain_type= "stuff",
    retriever = db.as_retriever()
)

Here, the `stuff` chain type is used. It is typically used for small documents and only few are passed in at most calls. It takes a list of documents, inserts them into prompts and sends them to LLM.

In [29]:
from langchain.agents import initialize_agent, Tool, AgentType

In [30]:
tools = [
    Tool(
        name = "Retrieval QA system",
        func= retrieval_qa.run,
        description= "Useful for answering questions"
    ),
]

agent = initialize_agent(
    tools,
    model,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [31]:
response = agent.run("Which battle is considered the greatest victory for Napoleon?")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


 I should research Napoleon's battles
Action: Retrieval QA system
Action Input: Napoleon's greatest victory
Observation:  The Battle of Austerlitz
Thought: I now know the final answer
Final Answer: The Battle of Austerlitz is considered the greatest victory for Napoleon.

> Finished chain.


In [32]:
print(response)

The Battle of Austerlitz is considered the greatest victory for Napoleon.


## Agents 
Agents are components which use LLMs to determine the suitable action to take along with the order. 

> Action : Using a tool, observing output or returning it to user

> Tool : Functions that perform specific duties such as Google Search, Database Lookups.

There are many types of agents:
1. `zero-shot-description` - Uses `ReAct` framework to decide best tool based on tool's description
2. `react-docstore` - Goes to docstore and does two things: Search (Search tool) and Lookup (Lookup tool)
3. `self-ask-with-search` - Intermediate Answer tool which is capable of looking up factual answers to queries
4. `conversational-react-description` - Remembers past conversations and responds

In [4]:
# Imports
from langchain.llms import OpenAI
from langchain.agents import AgentType, load_tools, initialize_agent, Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [5]:
model = OpenAI(model_name = "text-davinci-003", temperature = 0)

In [6]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [7]:
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID

In [9]:
!pip install google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.7 MB/s eta 0:00:00


In [11]:
search = GoogleSearchAPIWrapper()

In [12]:
tools = [
    Tool(
        name = "google-search",
        func = search.run,
        description= "Useful for searching google to answer current events"
    ),
]

In [13]:
agent = initialize_agent(
    tools, 
    model,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    max_iterations = 6
)

In [15]:
response = agent("What is the latest news about Twitter?")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


 I should look for recent news articles about Twitter
Action: google-search
Action Input: "Twitter news"
Observation: @BBCWorld. News, features and analysis from the World's newsroom. Breaking news, follow. @BBCBreaking . UK news,. @BBCNews . Latest sports news. 1 day ago ... Twitter has officially rebranded to "X" after owner Elon Musk changed ... head of ad sales for NBCUniversal, the parent company of NBC News. BREAKING: JOE BIDEN WINS Joe Biden will be the 46th president of the United States, CNN projects, after a victory in Pennsylvania puts the Scranton-born ... #WorldCup on Twitter: The G.O.A.T. ... Product Twitter Blue is back. And gold checkmarks are here! ... X Corp. About the company · Twitter for Good · Company news ... NEW NEWS! We've got a Kelce Bowl. #SBLVII. Image. New Heights and 2 others. 1,546. 10.7K. 64.3K. 9.1M · Square profile picture. 9 hours ago ... Elon Musk, who has long had a fascination with the letter X, is taking another step in turning the social platform

In [16]:
print(response)

{'input': 'What is the latest news about Twitter?', 'output': 'The latest news about Twitter includes the rebranding of the social platform to "X" by owner Elon Musk, the launch of Twitter Blue and Gold Checkmarks, and the capture of a new view of Jupiter in infrared light by NASAWebb.'}
